# Chelsea game predictor

This model only predicts porbability of chelsea winning any game. Data was cleaned, preprocessed and saved as chelsea_games sheet in Data/match_results file.

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [40]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from IPython.display import display

# Load the data
df = pd.read_excel('Data/arsenal_games.xlsx', sheet_name='arsenal')

# Filter the dataframe to include only played matches
df = df[df.match_status == "Played"]

# One-hot encode categorical variables
categorical_columns = ['against', 'game_type']
df_encoded = pd.get_dummies(df, columns=categorical_columns)

# Mapping result to numerical values
result_mapping = {'Win': 2, 'Draw': 1, 'Loss': 0}
df_encoded['result'] = df_encoded['result'].map(result_mapping)

# Selecting features and target variable
X = df_encoded.drop(columns=['team', 'date', 'result', 'venue', 'covid', 'match_status'])
y = df_encoded['result']

# Splitting the data with stratified sampling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# Initialize the model
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Predicting the results
y_prob = model.predict_proba(X_test)

# Extracting the probability of win, draw, and loss
win_probabilities = y_prob[:, 2]
draw_probabilities = y_prob[:, 1]
loss_probabilities = y_prob[:, 0]

# Add probabilities to the original test set
original_test = df.iloc[X_test.index].reset_index(drop=True)
original_test['win_probability'] = win_probabilities
original_test['draw_probability'] = draw_probabilities
original_test['loss_probability'] = loss_probabilities

# Display the results with probabilities
original_test_with_probabilities = original_test[['team', 'against', 'date', 'match_status', 'game_type', 'venue', 'scored_goals', 'conceded_goals', 'result', 'win_probability', 'draw_probability', 'loss_probability']]
display(original_test_with_probabilities)

# Confusion matrix
y_pred = model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
display(cm)

# Ensure the labels in the classification report match the unique labels in y_test
target_names = ['Loss', 'Draw', 'Win']

# Classification report
report = classification_report(y_test, y_pred, target_names=target_names)
print(report)


,team,against,date,match_status,game_type,venue,scored_goals,conceded_goals,result,win_probability,draw_probability,loss_probability
0,Arsenal,Brighton,9/29/2019,Played,Home,Meadow Park,4.0,0.0,Win,1.00,0.00,0.00
1,Arsenal,Reading,2022-02-03 00:00:00,Played,Away,Meadow Park,4.0,0.0,Win,1.00,0.00,0.00
2,Arsenal,West Ham,2022-08-05 00:00:00,Played,Home,Chigwell Construction Stadium,2.0,0.0,Win,0.95,0.05,0.00
3,Arsenal,Leicester City,2023-05-05 00:00:00,Played,Home,Meadow Park,1.0,0.0,Win,0.79,0.19,0.02
4,Arsenal,Everton,12/15/2019,Played,Away,Walton Hall Park,3.0,1.0,Win,0.99,0.01,0.00
5,Arsenal,Aston Villa,2022-01-05 00:00:00,Played,Home,Meadow Park,7.0,0.0,Win,0.95,0.05,0.00
6,Arsenal,Manchester Utd,2020-08-11 00:00:00,Played,Home,Leigh Sports Village Stadium,0.0,1.0,Loss,0.23,0.43,0.34
7,Arsenal,Manchester Utd,4/19/2023,Played,Away,Leigh Sports Village Stadium,0.0,1.0,Loss,0.18,0.58,0.24
8,Arsenal,Aston Villa,2022-11-12 00:00:00,Played,Away,Villa Park,4.0,1.0,Win,0.96,0.04,0.00
9,Arsenal,West Ham,11/26/2023,Played,Home,Meadow Park,3.0,0.0,Win,0.95,0.05,0.00


array([[ 3,  2,  0],
       [ 0,  2,  1],
       [ 0,  0, 23]], dtype=int64)

              precision    recall  f1-score   support

        Loss       1.00      0.60      0.75         5
        Draw       0.50      0.67      0.57         3
         Win       0.96      1.00      0.98        23

    accuracy                           0.90        31
   macro avg       0.82      0.76      0.77        31
weighted avg       0.92      0.90      0.90        31



In [32]:
df.result.value_counts()

result
Win     73
Loss    17
Draw    11
Name: count, dtype: int64